# part 5 데이터 누락 처리

In [ ]:
# 1. 누락 데이터 처리
# -- 누락 데이터 확인
# -- count()
# -- value_counts()
# -- isnull()  -> isnull().count()


In [2]:
# %load fonts.py
#!/usr/bin/env python

# In[ ]:


# matplotlib 한글 폰트 문제 해결
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
import platform

if platform.system() == 'Darwin':         # mac
    plt.rc('font',family='AppleGothic')
elif platform.system() == 'Windows':
    rc('font',family='Malgun Gothic')
plt.rc('axes',unicode_minus=False)



In [3]:
df = sns.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
# deck 컬럼의 값들의 개수 : value_counts()
df['deck'].value_counts(dropna=False)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

In [9]:
# isnull() : 누락 데이터 개수 확인
df.isnull().sum(axis=0)     # age:177,deck:688,embark_down:2,embarked:2

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [11]:
# 누락 데이터가 ()개 이상인 경우 컬럼을 삭제
# dropna() null값 있는 라인 통째로 삭제
df_1 = df.dropna(axis=1, thresh=500)
df_1.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [16]:
df_2 = df.dropna(subset=['deck','age'],how='any',axis=0)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 1 to 889
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     184 non-null    int64   
 1   pclass       184 non-null    int64   
 2   sex          184 non-null    object  
 3   age          184 non-null    float64 
 4   sibsp        184 non-null    int64   
 5   parch        184 non-null    int64   
 6   fare         184 non-null    float64 
 7   embarked     182 non-null    object  
 8   class        184 non-null    category
 9   who          184 non-null    object  
 10  adult_male   184 non-null    bool    
 11  deck         184 non-null    category
 12  embark_town  182 non-null    object  
 13  alive        184 non-null    object  
 14  alone        184 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 18.4+ KB


In [28]:
# 누락 데이터 치환 : df.fillna(method='ffill') ffill , bfill, 값
# age 컬럼에 Nan 을 age의 평균값으로 치환
# 가장 빈번하게 나오는 값으로 치환
# 가장 큰 값으로 치환, 가장 작은 값으로 치환
# 이전 값으로 치환, 이후 나오는 값으로 치환  fillna(method='ffill'or'bfill')
df_ = df.copy()
df_.age.isnull().value_counts()
# df_['age'] = df_['age'].fillna(df['age'].mean(axis=0))
df_['age'].fillna(0,inplace=True)

df_.age.isnull().value_counts()
df_.age

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     0.0
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64

In [44]:
print(df['embark_town'][825:830])       #series 반환
# embark_town의 Nan-> 이전 값으로 대체
df_2 = df.copy()
df_2['embark_town'].fillna(method='ffill', inplace=True)
df_2.embark_town[825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object

In [47]:
print(df['embark_town'][825:830])       #series 반환
# embark_town의 Nan-> 이후 값으로 대체
df_2 = df.copy()
df_2['embark_town'].fillna(method='bfill', inplace=True)
df_2.embark_town[825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829      Cherbourg
Name: embark_town, dtype: object

In [46]:
# embark_town에 Nan -> 가장 빈번하게 나오는 값으로 대체
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()
df_2 = df.copy()
df_2['embark_town'].fillna(most_freq, inplace=True)
df_2.embark_town[825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object

In [91]:
# 누락 데이터가 Nan이 아닌 경우 : 특수 문자인 경우
# 특수 문자를 Nan으로 치환 후 Nan 처리
df = pd.read_csv('./dataset/auto-mpg.csv',header=None)
df


df.columns =  ['mpg','cylinders','displacement','horsepower','weight',
             'acceleration','model year','origin','name']

df.info()
df

,0,1,2,3,4,5,6,7,8
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger


In [93]:
# df['horsepower'].astypeype(float)
df['horsepower'].unique()  # ? 문자발견

array(['130.0', '165.0', '150.0', '140.0', '198.0', '220.0', '215.0',
       '225.0', '190.0', '170.0', '160.0', '95.00', '97.00', '85.00',
       '88.00', '46.00', '87.00', '90.00', '113.0', '200.0', '210.0',
       '193.0', '?', '100.0', '105.0', '175.0', '153.0', '180.0', '110.0',
       '72.00', '86.00', '70.00', '76.00', '65.00', '69.00', '60.00',
       '80.00', '54.00', '208.0', '155.0', '112.0', '92.00', '145.0',
       '137.0', '158.0', '167.0', '94.00', '107.0', '230.0', '49.00',
       '75.00', '91.00', '122.0', '67.00', '83.00', '78.00', '52.00',
       '61.00', '93.00', '148.0', '129.0', '96.00', '71.00', '98.00',
       '115.0', '53.00', '81.00', '79.00', '120.0', '152.0', '102.0',
       '108.0', '68.00', '58.00', '149.0', '89.00', '63.00', '48.00',
       '66.00', '139.0', '103.0', '125.0', '133.0', '138.0', '135.0',
       '142.0', '77.00', '62.00', '132.0', '84.00', '64.00', '74.00',
       '116.0', '82.00'], dtype=object)

In [94]:
# ? 를 Nan으로 치환

df['horsepower'].replace('?',np.nan,inplace=True)
df['horsepower'].astype(float)

# horsepower 값이 Nan이면 해당 행 삭제
df.dropna(subset='horsepower',axis=0,inplace=True)
df['horsepower']=df['horsepower'].astype(float)

In [95]:
df['horsepower'].unique()

array([130., 165., 150., 140., 198., 220., 215., 225., 190., 170., 160.,
        95.,  97.,  85.,  88.,  46.,  87.,  90., 113., 200., 210., 193.,
       100., 105., 175., 153., 180., 110.,  72.,  86.,  70.,  76.,  65.,
        69.,  60.,  80.,  54., 208., 155., 112.,  92., 145., 137., 158.,
       167.,  94., 107., 230.,  49.,  75.,  91., 122.,  67.,  83.,  78.,
        52.,  61.,  93., 148., 129.,  96.,  71.,  98., 115.,  53.,  81.,
        79., 120., 152., 102., 108.,  68.,  58., 149.,  89.,  63.,  48.,
        66., 139., 103., 125., 133., 138., 135., 142.,  77.,  62., 132.,
        84.,  64.,  74., 116.,  82.])

In [97]:
# 중복 데이터 처리
# 중복 데이터를 갖는 데이터프레임 만들기
df = pd.DataFrame({'c1':['a', 'a', 'b', 'a', 'b'],
                  'c2':[1, 1, 1, 2, 2],
                  'c3':[1, 1, 2, 2, 2]})
df.duplicated()   # 이전에 완전히 같은 행이 존재할 경우 True

0    False
1     True
2    False
3    False
4    False
dtype: bool

In [105]:
# c2 컬럼에 대해서 중복 체크
df['c2'].duplicated()

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool

In [106]:
# 중복 데이터 제거하는 함수 : drop_duplicates()
df2 = df.drop_duplicates()
df2

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [109]:
# 중복 데이터 제거하는 함수 : drop_duplicates()
df2 = df.drop_duplicates(subset=['c2','c3'])
df2

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


In [110]:
df2 = df.drop_duplicates(subset=['c1'])
df2

,c1,c2,c3
0,a,1,1
2,b,1,2


In [ ]:
# 데이터 정제
# 1. 누락 데이터 처리 -> Nan, 특수문자
#   dropna(subset=[],thresh=,axis=), 
#   fillna(값, method='ffill','bfill')
# 2. 중복 데이터 처리
#   drop_duplicates(subset=[])
# 3. 데이터 표준화
#    1) 단위환산
#    2) 자료형 변화
# 4. 범주형 데이터 처리 : 카테고리형
#    구간으로 데이터를 나누어서 처리
# 5. 더미 변수 : 카테고리 데이터인 경우 0와1로 값을 변환
# 6. train과 test 분리 -> 모델 학습

In [112]:
# 단위 환산
df = pd.read_csv('./dataset/auto-mpg.csv',header=None)

df.columns =  ['mpg','cylinders','displacement','horsepower','weight',
             'acceleration','model year','origin','name']
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite


In [113]:
# mpg(mile per gallon) -> kpl
df['kpl'] = (df['mpg'] * 0.425144).round(2)
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.65


In [146]:
# 자료형 변환
# df.dtypes
# df['horsepower']
# df['horsepower'] = df['horsepower'].astype(float)

df['horsepower'].unique()  #  '?' -> Nan 치환, Nan 처리
df['horsepower'].replace('?',np.nan,inplace=True)
df['horsepower'] = df['horsepower'].astype(float)
df.dtypes

mpg              float64
cylinders          int64
displacement     float64
horsepower       float64
weight           float64
acceleration     float64
model year      category
origin          category
name              object
kpl              float64
dtype: object

In [125]:
# origin 컬럼의 정보 분석
df['origin'].dtype
df['origin'].unique()

# int형을 범주형으로 변환
df['origin'] = df['origin'].astype('category')
df['origin'].dtype

# 카테고리값을 문자형으로 변경 1:USA,2:EU,3:JPN
df['origin'].replace({1:'USA',2:'EU',3:'JPN'},inplace=True)
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.65
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.80
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.23
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,USA,ford mustang gl,11.48
394,44.0,4,97.0,52.0,2130.0,24.6,82,EU,vw pickup,18.71
395,32.0,4,135.0,84.0,2295.0,11.6,82,USA,dodge rampage,13.60
396,28.0,4,120.0,79.0,2625.0,18.6,82,USA,ford ranger,11.90


In [ ]:
# 범주형 자료의 데이터 타입을 'str'로 변경
df['origin'] = df['origin'].astype('str')


In [141]:
df['origin'].sample(3)

23      EU
210    JPN
336    USA
Name: origin, dtype: category
Categories (3, object): ['USA', 'EU', 'JPN']

In [143]:
# model year 컬럼을 분석 -> 카테고리형으로 변형
df['model year'].value_counts()

df['model year'] = df['model year'].astype('category')
df['model year'].dtype

CategoricalDtype(categories=[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82], ordered=False)

In [151]:
# 문자 또는 숫자를 카테고리형으로 변경
# 원-핫 인코딩을 통해 자료 변형
# 연속형 자료를 범주형으로 변경
# 1. horsepower 컬럼을 3구간으로 나누어서 범주형으로 나누기
# 2. 각 구간별 경계선의 값 필요 : np.histogram(컬럼명, bins=구간수)
#     각 구간별 경계값과 데이터 개수를 튜플로 반환
df.dropna(subset=['horsepower'],axis=0,inplace=True)
count, bin_dividers = np.histogram(df['horsepower'],bins=3)

bin_labels = ['저출력','보통출력','고출력']

# cut()함수를 사용하여 각 값들을 bin에 할당
df['hp_bin']=pd.cut(x=df['horsepower'],
                    bins=bin_dividers,
                    labels=bin_labels,
                   include_lowest=True)

df[['horsepower','hp_bin']].head(10)

,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


In [154]:
# 더미 변수
pd.get_dummies(df['origin'],prefix='origin')
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl,hp_bin
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.65,보통출력
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.38,보통출력
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.65,보통출력
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.80,보통출력
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.23,보통출력
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,USA,ford mustang gl,11.48,저출력
394,44.0,4,97.0,52.0,2130.0,24.6,82,EU,vw pickup,18.71,저출력
395,32.0,4,135.0,84.0,2295.0,11.6,82,USA,dodge rampage,13.60,저출력
396,28.0,4,120.0,79.0,2625.0,18.6,82,USA,ford ranger,11.90,저출력


In [155]:
titanic = sns.load_dataset('titanic')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [167]:
titanic['alive']   # object를 category로 변경
titanic['alive'] = titanic['alive'].astype('category')
alive_dummy = pd.get_dummies(titanic['alive'],prefix='alive',drop_first=True)
pd.concat([titanic,alive_dummy],axis=1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,alive_yes
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,1
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,1
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,0
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,1
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,0
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,1
